
# Data Preparation: Video Game Sales

## Tujuan
Menyiapkan dataset vgsales menjadi data siap analisis untuk:
- Market structure.
- Regional preference.
- Platform lifecycle.

Catatan metodologi:
- Year adalah tahun rilis, bukan time series penjualan.
- Analisis YoY tidak digunakan.
- Fokus pada cohort dan lifecycle.

----
## Load Data

In [1]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv("https://raw.githubusercontent.com/AlvitoDwiP/video-game-sales/refs/heads/main/data/raw/vgsales.csv")
df.shape


(16598, 11)

Dataset berisi penjualan kumulatif per game dan region. Analisis difokuskan pada struktur pasar dan perbandingan relatif.

### Drop baris tanpa identitas utama
Game tanpa nama atau platform tidak bisa dipakai untuk analisis entitas.

In [ ]:
df = df.dropna(subset=["Name", "Platform"])
df.shape


### Standardisasi kolom waktu
Kolom Year dikonversi ke numerik untuk memungkinkan cohort rilis.

In [ ]:
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df["Year"].isna().mean()


### Penanganan kolom sales
Missing sales diperlakukan sebagai nol agar agregasi tidak menghasilkan NaN.

In [ ]:
sales_cols = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]
df[sales_cols] = df[sales_cols].fillna(0)
df[sales_cols].isna().sum()


### Penanganan kategori
Genre dan Publisher yang hilang tetap dipertahankan sebagai kategori eksplisit.

In [ ]:
df["Genre"] = df["Genre"].fillna("Unknown")
df["Publisher"] = df["Publisher"].fillna("Unknown")

df[["Genre", "Publisher"]].isna().sum()


### Pembuatan total sales
Total_Sales dipakai sebagai basis perbandingan lintas entitas.

In [ ]:
df["Total_Sales"] = df[sales_cols].sum(axis=1)
df["Total_Sales"].describe()


### Perhitungan share regional
Share digunakan untuk membandingkan preferensi pasar tanpa bias ukuran total

In [ ]:
df["Share_NA"] = df["NA_Sales"] / df["Total_Sales"].replace(0, np.nan)
df["Share_EU"] = df["EU_Sales"] / df["Total_Sales"].replace(0, np.nan)
df["Share_JP"] = df["JP_Sales"] / df["Total_Sales"].replace(0, np.nan)

df[["Share_NA", "Share_EU", "Share_JP"]] = df[["Share_NA", "Share_EU", "Share_JP"]].fillna(0)

df[["Share_NA", "Share_EU", "Share_JP"]].head()


### Cohort rilis
Cohort dekade dipakai sebagai pengganti tren tahunan yang tidak valid.

In [ ]:
df["decade_cohort"] = (df["Year"] // 10 * 10).astype("Int64")
df[["Year", "decade_cohort"]].dropna().head()


### Save output
Dataset ini menjadi input utama untuk seluruh analisis berikutnya.